# Configuration

NOTES: The warnings after the import are referred to the fact that Tensorflow 2.x versions are built to directly look for a GPU in the system. The warning can be forgot if you are not going to use the GPU. 

**WHAT CHANGES FROM EXPERIMENT 1**:
- In feature_extractor function we have also the function to add the information about the energy --> the size of the input feature changes
- So, change the input size of all the models change 

In [1]:
!source myenv/bin/activate

In [2]:
# samples in 5 seconds of audio, 16 KHz sample rate 
LENGTH_CHOSEN =  80000

In [3]:
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set_style('whitegrid')
import IPython.display as ipd
import librosa.display
import numpy as np
import pickle
import scipy
import ipywidgets
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold


from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# from livelossplot import PlotLossesKeras
tf.config.list_physical_devices('GPU')

2021-09-29 15:20:15.081188: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-29 15:20:15.081211: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-29 15:20:16.373480: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


[]

2021-09-29 15:20:16.375355: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-29 15:20:16.447470: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-29 15:20:16.447511: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /proc/driver/nvidia/version does not exist


# Utils

In [4]:
def load_files(df):
    X = []
    for i in tqdm(df['path']): 
        X.append(librosa.load(i, res_type='kaiser_fast', sr=16000))
    return X

def extract_samples(X): 
    samples = []
    for ind,i in enumerate(X):
        samples.append(i[0])
    return samples 

def extract_labels(df): 
    labels = df['emotion_label'].copy()
    return labels 

def compute_lengths(samples): 
    lengths = [len(x) for x in samples]
    return lengths 

def check_outliers(lengths):
    # outliers
    lengths = np.array(lengths)
    print((lengths > 300000).sum())
    new_lengths = lengths[lengths < 300000]
    return new_lengths 

def compute_mean_length(lengths): 
    return lengths.mean()


def cut_and_pad(samples, labels, length_chosen = LENGTH_CHOSEN): 
    X_new = []
    y_new = []
    count = 0 
    for ind,i in enumerate(samples):
        if i.shape[0] < 300000:
            if i.shape[0] > length_chosen:
                new = i[:length_chosen]
                X_new.append(new)
            elif i.shape[0] < length_chosen:
                new = np.pad(i,math.ceil((length_chosen-i.shape[0])/2), mode='median')
                X_new.append(new)
            else:
                X_new.append(i)
            y_new.append(labels[count])
        count+=1
    
    return X_new, y_new
    
def compute_mfccs(samples, n_mfcc): 
    mfccs = []
    for i in tqdm(samples):
        mfcc = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=n_mfcc)
        mfcc = mfcc.T
        mfcc = np.array(mfcc)
        mfccs.append(mfcc[:, 1:]) # get rid of the first component 
    mfccs = np.array(mfccs)
    return mfccs

def compute_energy(samples): 
    energy_per_sample = []
    for i in tqdm(samples):
        energy = librosa.feature.rms(i)
        energy = energy.T 
        energy = np.array(energy)
        energy_per_sample.append(energy) 
    return energy_per_sample
       
def feature_extractor(df_train, df_val, df_test, n_mfcc): 
    load_train = load_files(df_train)
    samples_train = extract_samples(load_train)
    labels_train = extract_labels(df_train)
    samples_train, labels_train = cut_and_pad(samples_train, labels_train)
    samples_train = np.array(samples_train)
    labels_train = np.array(labels_train)
    mfccs_train = compute_mfccs(samples_train, n_mfcc = n_mfcc)
    # energy 
    energy_train = compute_energy(samples_train) 
    features_train = []
    for i in range(len(mfccs_train)): 
        if len(mfccs_train) == len(energy_train): 
            conc = np.concatenate((mfccs_train[i], energy_train[i]), axis = 1)
            features_train.append(conc)

    
    
  
    load_val = load_files(df_val)
    samples_val = extract_samples(load_val)
    labels_val = extract_labels(df_val)
    samples_val, labels_val = cut_and_pad(samples_val, labels_val)
    samples_val = np.array(samples_val)
    labels_val = np.array(labels_val)
    mfccs_val = compute_mfccs(samples_val, n_mfcc = n_mfcc)
    # energy 
    energy_val = compute_energy(samples_val) 
    features_val = []
    for i in range(len(mfccs_val)): 
        if len(mfccs_val) == len(energy_val): 
            conc = np.concatenate((mfccs_val[i], energy_val[i]), axis = 1)
            features_val.append(conc)
    
    
    
    
    load_test = load_files(df_test)
    samples_test = extract_samples(load_test)
    labels_test = extract_labels(df_test)
    samples_test, labels_test = cut_and_pad(samples_test, labels_test)
    samples_test = np.array(samples_test)
    labels_test = np.array(labels_test)
    mfccs_test = compute_mfccs(samples_test, n_mfcc = n_mfcc)
    # energy 
    energy_test = compute_energy(samples_test) 
    features_test=[]
    for i in range(len(mfccs_test)): 
        if len(mfccs_test) == len(energy_test): 
            conc = np.concatenate((mfccs_test[i], energy_test[i]), axis = 1)
            features_test.append(conc)
    

    return np.array(features_train), labels_train,  np.array(features_val), labels_val, np.array(features_test), labels_test
    

def feature_extractor_tess(df_train,  df_test, n_mfcc): 
    # we do not have the validation set here 
    load_train = load_files(df_train)
    samples_train = extract_samples(load_train)
    labels_train = extract_labels(df_train)
    samples_train, labels_train = cut_and_pad(samples_train, labels_train)
    samples_train = np.array(samples_train)
    labels_train = np.array(labels_train)
    mfccs_train = compute_mfccs(samples_train, n_mfcc = n_mfcc)
    # energy 
    energy_train = compute_energy(samples_train) 
    features_train = []
    for i in range(len(mfccs_train)): 
        if len(mfccs_train) == len(energy_train): 
            conc = np.concatenate((mfccs_train[i], energy_train[i]), axis = 1)
            features_train.append(conc)

    
    
    load_test = load_files(df_test)
    samples_test = extract_samples(load_test)
    labels_test = extract_labels(df_test)
    samples_test, labels_test = cut_and_pad(samples_test, labels_test)
    samples_test = np.array(samples_test)
    labels_test = np.array(labels_test)
    mfccs_test = compute_mfccs(samples_test, n_mfcc = n_mfcc)
    # energy 
    energy_test = compute_energy(samples_test) 
    features_test=[]
    for i in range(len(mfccs_test)): 
        if len(mfccs_test) == len(energy_test): 
            conc = np.concatenate((mfccs_test[i], energy_test[i]), axis = 1)
            features_test.append(conc)
    
    
    return np.array(features_train), labels_train, np.array(features_test), labels_test
    
def encode_labels(labels_train, labels_val, labels_test): 
    
    emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
    y_train = pd.Series(labels_train).replace(emotion_enc)
  
    y_test = pd.Series(labels_test).map(emotion_enc)
    y_val = pd.Series(labels_val).map(emotion_enc)
    return y_train, y_val, y_test 


def encode_labels_tess(labels_train, labels_test): 
    
    emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}
    y_train = pd.Series(labels_train).replace(emotion_enc)
  
    y_test = pd.Series(labels_test).map(emotion_enc)
    return y_train, y_test
    
def standard_scaling(X_train, X_val, X_test): 
  
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
    return X_train, X_val, X_test, scaler 
    
def standard_scaling_tess(X_train, X_test): 
  
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    return X_train, X_test, scaler   
    


# Compute dataframes for datasets and split in Train, Val, Test 

In [5]:
main_path = '/media/helemanc/OS/Users/i2CAT/Desktop/Datasets SER/'
TESS = os.path.join(main_path, "tess/TESS Toronto emotional speech set data/") 
RAV = os.path.join(main_path, "ravdess-emotional-speech-audio/audio_speech_actors_01-24")
SAVEE = os.path.join(main_path, "savee/ALL/")
CREMA = os.path.join(main_path, "creamd/AudioWAV/")

## RADVESS

In [6]:
lst = []
emotion = []
voc_channel = []
full_path = []
modality = []
intensity = []
actors = []
phrase =[]

for root, dirs, files in tqdm(os.walk(RAV)):
    for file in files:
        try:
            #Load librosa array, obtain mfcss, store the file and the mfcss information in a new array
            # X, sample_rate = librosa.load(os.path.join(root,file), res_type='kaiser_fast')
            # mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
            # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
            # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
           
            modal = int(file[1:2])
            vchan = int(file[4:5])
            lab = int(file[7:8])
            ints = int(file[10:11])
            phr = int(file[13:14])
            act = int(file[18:20])
            # arr = mfccs, lab
            # lst.append(arr)
            
            modality.append(modal)
            voc_channel.append(vchan)
            emotion.append(lab) #only labels
            intensity.append(ints)
            phrase.append(phr)
            actors.append(act)
            
            full_path.append((root, file)) # only files
          # If the file is not valid, skip it
        except ValueError:
            continue

25it [00:00, 1636.43it/s]


In [7]:
# 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# merge neutral and calm
emotions_list = ['neutral', 'neutral', 'happy', 'sadness', 'angry', 'fear', 'disgust', 'surprise']
emotion_dict = {em[0]+1:em[1] for em in enumerate(emotions_list)}

df = pd.DataFrame([emotion, voc_channel, modality, intensity, actors, actors,phrase, full_path]).T
df.columns = ['emotion', 'voc_channel', 'modality', 'intensity', 'actors', 'gender', 'phrase', 'path']
df['emotion'] = df['emotion'].map(emotion_dict)
df['voc_channel'] = df['voc_channel'].map({1: 'speech', 2:'song'})
df['modality'] = df['modality'].map({1: 'full AV', 2:'video only', 3:'audio only'})
df['intensity'] = df['intensity'].map({1: 'normal', 2:'strong'})
df['actors'] = df['actors']
df['gender'] = df['actors'].apply(lambda x: 'female' if x%2 == 0 else 'male')
df['phrase'] = df['phrase'].map({1: 'Kids are talking by the door', 2:'Dogs are sitting by the door'})
df['path'] = df['path'].apply(lambda x: x[0] + '/' + x[1])

In [8]:
# remove files with noise to apply the same noise to all files for data augmentation 
df = df[~df.path.str.contains('noise')]

In [9]:
df.head()

,emotion,voc_channel,modality,intensity,actors,gender,phrase,path
0,disgust,speech,audio only,normal,1,male,Dogs are sitting by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,disgust,speech,audio only,strong,1,male,Kids are talking by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,disgust,speech,audio only,strong,1,male,Kids are talking by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
6,disgust,speech,audio only,strong,1,male,Dogs are sitting by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
8,disgust,speech,audio only,strong,1,male,Dogs are sitting by the door,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [10]:
# only speech
RAV_df = df
RAV_df = RAV_df.loc[RAV_df.voc_channel == 'speech']

In [11]:
RAV_df.insert(0, "emotion_label", RAV_df.emotion, True)

In [12]:
RAV_df = RAV_df.drop(['emotion', 'voc_channel', 'modality', 'intensity', 'phrase'], 1)

In [13]:
RAV_df

,emotion_label,actors,gender,path
0,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
6,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
8,disgust,1,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
...,...,...,...,...
2871,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2873,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2875,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2877,neutral,24,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [14]:
RAV_train = []
RAV_val = []
RAV_test = []

In [15]:
for index, row in RAV_df.iterrows():
    if row['actors'] in range(1,21): 
        RAV_train.append(row) 
    elif row['actors'] in range(21,23): 
        RAV_val.append(row)
    elif row['actors'] in range(23,25): 
        RAV_test.append(row)
len(RAV_train), len(RAV_val), len(RAV_test)

(1200, 120, 120)

In [16]:
RAV_train = pd.DataFrame(RAV_train)
RAV_val = pd.DataFrame(RAV_val)
RAV_test = pd.DataFrame(RAV_test)

In [17]:
RAV_train = RAV_train.drop(['actors'], 1)
RAV_val = RAV_val.drop(['actors'], 1)
RAV_test = RAV_test.drop(['actors'], 1)

In [18]:
RAV_train.reset_index(drop=True, inplace = True) 
RAV_val.reset_index(drop=True, inplace = True) 
RAV_test.reset_index(drop=True, inplace = True ) 

## SAVEE

In [19]:
# Get the data location for SAVEE
dir_list = os.listdir(SAVEE)

# parse the filename to get the emotions
emotion=[]
path = []
actors = []
gender = []
for i in dir_list:
    actors.append(i[:2])
    if i[-8:-6]=='_a':
        emotion.append('angry')
        gender.append('male')
    elif i[-8:-6]=='_d':
        emotion.append('disgust')
        gender.append('male')
    elif i[-8:-6]=='_f':
        emotion.append('fear')
        gender.append('male')
    elif i[-8:-6]=='_h':
        emotion.append('happy')
        gender.append('male')
    elif i[-8:-6]=='_n':
        emotion.append('neutral')
        gender.append('male')
    elif i[-8:-6]=='sa':
        emotion.append('sadness')
        gender.append('male')
    elif i[-8:-6]=='su':
        emotion.append('surprise')
        gender.append('male') 
    else:
        emotion.append('Unknown') 
    path.append(SAVEE + i)
    
# Now check out the label count distribution 
SAVEE_df = pd.DataFrame(emotion, columns = ['emotion_label'])
                      
SAVEE_df = pd.concat([SAVEE_df,
                      pd.DataFrame(actors, columns = ['actors']),
                      pd.DataFrame(gender, columns = ['gender']), 
                      pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_df.emotion_label.value_counts()

neutral     120
sadness      60
surprise     60
happy        60
disgust      60
fear         60
angry        60
Name: emotion_label, dtype: int64

In [20]:
SAVEE_df.head()

,emotion_label,actors,gender,path
0,neutral,DC,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
1,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
3,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,sadness,KL,male,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [21]:
SAVEE_train = []
SAVEE_val = []
SAVEE_test = []

In [22]:
#DC, JE, JK, KL
for index, row in SAVEE_df.iterrows(): 
    if row['actors'] == 'DC' or row ['actors'] == 'JE':
        SAVEE_train.append(row)
    elif row['actors'] == 'JK': 
        SAVEE_val.append(row)
    else: 
        SAVEE_test.append(row)
len(SAVEE_train), len(SAVEE_val), len(SAVEE_test)

(240, 120, 120)

In [23]:
SAVEE_train = pd.DataFrame(SAVEE_train)
SAVEE_val = pd.DataFrame(SAVEE_val)
SAVEE_test = pd.DataFrame(SAVEE_test)

In [24]:
SAVEE_train = SAVEE_train.drop(['actors'], 1)
SAVEE_val = SAVEE_val.drop(['actors'], 1)
SAVEE_test = SAVEE_test.drop(['actors'], 1)

In [25]:
SAVEE_train = SAVEE_train.reset_index(drop=True) 
SAVEE_val = SAVEE_val.reset_index(drop=True) 
SAVEE_test = SAVEE_test.reset_index(drop=True) 

## TESS

In [26]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

path = []
emotion = []
gender = []
actors = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry':
            emotion.append('angry')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_angry': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_disgust' :
            emotion.append('disgust')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_disgust': 
            emotion.append('disgust')
            gender.append('female')
            actors.append('YAF')
            
            
        elif i == 'OAF_Fear':
            emotion.append('fear')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_fear': 
            emotion.append('fear')
            gender.append('female')
            actors.append('YAF') 
            
            
        elif i == 'OAF_happy' :
            emotion.append('happy')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_happy': 
            emotion.append('angry')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_neutral':
            emotion.append('neutral')
            gender.append('female')
            actors.append('OAF')   
        elif i == 'YAF_neutral': 
            emotion.append('neutral')
            gender.append('female')
            actors.append('YAF')      
            
                
        elif i == 'OAF_Pleasant_surprise':
            emotion.append('surprise')
            gender.append('female')
            actors.append('OAF')
        
        elif i == 'YAF_pleasant_surprised': 
            emotion.append('surprise')
            gender.append('female')
            actors.append('YAF')            
            
        elif i == 'OAF_Sad':
            emotion.append('sadness')
            gender.append('female')
            actors.append('OAF')
        elif i == 'YAF_sad': 
            emotion.append('sadness')
            gender.append('female')
            actors.append('YAF')            
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['emotion_label'])
TESS_df = pd.concat([TESS_df, pd.DataFrame(gender, columns = ['gender']), 
                     pd.DataFrame(actors, columns= ['actors']),
                     pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.emotion_label.value_counts()

angry       1200
fear         800
surprise     800
sadness      800
disgust      800
neutral      800
happy        400
Name: emotion_label, dtype: int64

In [27]:
TESS_df= TESS_df[~TESS_df.path.str.contains('noise')]

In [28]:
TESS_train = []
TESS_test = []

In [29]:
for index, row in TESS_df.iterrows(): 
    if row['actors'] == 'YAF': 
        TESS_train.append(row)
    else: 
        TESS_test.append(row)
len(TESS_train), len(TESS_test)

(1400, 1400)

In [30]:
TESS_train = pd.DataFrame(TESS_train)
TESS_test = pd.DataFrame(TESS_test)

In [31]:
TESS_train = TESS_train.reset_index(drop=True) 
TESS_test  = TESS_test.reset_index(drop=True) 

## CREMA-D

In [32]:
males = [1,
5,
11,
14,
15,
16,
17,
19,
22,
23,
26,
27,
31,
32,
33,
34,
35,
36,
38,
39,
41,
42,
44,
45,
48,
50,
51,
57,
59, 
62, 
64,
65, 
66,
67,
68,
69,
70,
71,
77, 
80, 
81, 
83, 
85, 
86, 
87,
88, 
90]

In [33]:
females = [ 2,
3,
4,
6,
7,
8,
9,
10,
12,
13,
18,
20,
21,
24,
25,
28,
29,
30,
37,
40,
43,
46,
47,
49,
52,
53,
54,
55,
56, 
58, 
60,
61,
63,
72, 
73, 
74, 
75, 
76, 
78, 
79, 
82, 
84, 
89, 
91]

In [34]:
crema_directory_list = os.listdir(CREMA)

file_emotion = []
file_path = []
actors = []
gender = []




for file in crema_directory_list:

    # storing file emotions
    part=file.split('_')
    
    # use only high intensity files
    if "HI" in part[3] :
        actor = part[0][2:]
        actors.append(actor)
        if int(actor) in males:
            gender.append('male')
        else: 
            gender.append('female')
    
        # storing file paths
        file_path.append(CREMA + file)
        if part[2] == 'SAD':
            file_emotion.append('sadness')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['emotion_label'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['path'])
actors_df = pd.DataFrame(actors, columns=['actors'])
gender_df = pd.DataFrame(gender, columns=['gender'])                      
Crema_df = pd.concat([emotion_df, actors_df, gender_df, path_df], axis=1)
Crema_df.head()

,emotion_label,actors,gender,path
0,happy,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
1,sadness,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,angry,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
3,disgust,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,fear,91,female,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [35]:
Crema_df.shape

(455, 4)

In [36]:
actor_files = {}

for index, row in Crema_df.iterrows():
    actor = row['actors']
    if actor not in actor_files.keys(): 
        actor_files[actor] = 1
    else: 
        actor_files[actor]+=1

In [37]:
actor_files

{'91': 5,
 '90': 5,
 '89': 5,
 '88': 5,
 '87': 5,
 '86': 5,
 '85': 5,
 '84': 5,
 '83': 5,
 '82': 5,
 '81': 5,
 '80': 5,
 '79': 5,
 '78': 5,
 '77': 5,
 '76': 5,
 '75': 5,
 '74': 5,
 '73': 5,
 '72': 5,
 '71': 5,
 '70': 5,
 '69': 5,
 '68': 5,
 '67': 5,
 '66': 5,
 '65': 5,
 '64': 5,
 '63': 5,
 '62': 5,
 '61': 5,
 '60': 5,
 '59': 5,
 '58': 5,
 '57': 5,
 '56': 5,
 '55': 5,
 '54': 5,
 '53': 5,
 '52': 5,
 '51': 5,
 '50': 5,
 '49': 5,
 '48': 5,
 '47': 5,
 '46': 5,
 '45': 5,
 '44': 5,
 '43': 5,
 '42': 5,
 '41': 5,
 '40': 5,
 '39': 5,
 '38': 5,
 '37': 5,
 '36': 5,
 '35': 5,
 '34': 5,
 '33': 5,
 '32': 5,
 '31': 5,
 '30': 5,
 '29': 5,
 '28': 5,
 '27': 5,
 '26': 5,
 '25': 5,
 '24': 5,
 '23': 5,
 '22': 5,
 '21': 5,
 '20': 5,
 '19': 5,
 '18': 5,
 '17': 5,
 '16': 5,
 '15': 5,
 '14': 5,
 '13': 5,
 '12': 5,
 '11': 5,
 '10': 5,
 '09': 5,
 '08': 5,
 '07': 5,
 '06': 5,
 '05': 5,
 '04': 5,
 '03': 5,
 '02': 5,
 '01': 5}

In [38]:
count_males = 0 
count_females = 0 
male_list = []
for index, row in Crema_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1

In [39]:
count_males, count_females

(235, 220)

Since there are more males than females we will remove randomly 3 male actors (since there are exactly 5 audio files per actor)

In [40]:
import random 
random.seed(42)
males_to_remove = random.sample(male_list, 3)
males_to_remove

['17', '80', '88']

In [41]:
new_df = []
for index, row in Crema_df.iterrows(): 
    if row['actors'] not in males_to_remove: 
        new_df.append(row)

In [42]:
CREMA_df = pd.DataFrame(new_df)

In [43]:
for index, row in CREMA_df.iterrows(): 
    if row['actors'] == '17': 
        print("Elements not removed")

In [44]:
count_males = 0 
count_females = 0 
male_list = []
female_list = []
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if gender == 'male':
        count_males +=1
        if actor not in male_list: 
            male_list.append(actor)
    else: 
        count_females +=1
        if actor not in female_list: 
            female_list.append(actor)

In [45]:
count_males, count_females

(220, 220)

In [46]:
len(female_list)

44

In [47]:
len(male_list)

44

In [48]:
CREMA_train = []
CREMA_val = []
CREMA_test = []

In [49]:
females_train = random.sample(female_list, 32)
males_train = random.sample(male_list, 32)

# remove the elements assigned to train 
for element in females_train:
    if element in female_list:
        female_list.remove(element)
        
for element in males_train:
    if element in male_list:
        male_list.remove(element)

         
females_val = random.sample(female_list, 6) 
males_val = random.sample(male_list, 6) 

# remove the elements assigned to val
for element in females_val:
    if element in female_list:
        female_list.remove(element)
        
for element in males_val:
    if element in male_list:
        male_list.remove(element)
        
females_test = random.sample(female_list, 6) 
males_test = random.sample(male_list, 6)        

In [50]:
females_train, males_train, females_val, males_val, females_test, males_test

(['54',
  '56',
  '58',
  '74',
  '76',
  '13',
  '78',
  '29',
  '84',
  '89',
  '09',
  '60',
  '04',
  '55',
  '52',
  '91',
  '02',
  '07',
  '46',
  '49',
  '37',
  '10',
  '20',
  '75',
  '21',
  '53',
  '06',
  '28',
  '18',
  '63',
  '30',
  '03'],
 ['57',
  '69',
  '65',
  '45',
  '77',
  '81',
  '41',
  '15',
  '44',
  '23',
  '59',
  '86',
  '34',
  '01',
  '85',
  '66',
  '31',
  '33',
  '05',
  '48',
  '50',
  '67',
  '51',
  '22',
  '36',
  '87',
  '71',
  '39',
  '42',
  '11',
  '32',
  '14'],
 ['43', '61', '40', '47', '73', '24'],
 ['62', '68', '64', '83', '70', '26'],
 ['08', '79', '12', '25', '72', '82'],
 ['16', '19', '38', '35', '27', '90'])

In [51]:
train = females_train + males_train 
val = females_val + males_val 
test = females_test + males_test

In [52]:
for index, row in CREMA_df.iterrows(): 
    gender = row['gender']
    actor = row['actors']
    if actor in train: 
        CREMA_train.append(row)
    elif actor in val: 
        CREMA_val.append(row)
    else:
        CREMA_test.append(row)

In [53]:
CREMA_train = pd.DataFrame(CREMA_train) 
CREMA_val = pd.DataFrame(CREMA_val) 
CREMA_test = pd.DataFrame(CREMA_test)

In [54]:
CREMA_train.shape, CREMA_val.shape, CREMA_test.shape

((320, 4), (60, 4), (60, 4))

In [55]:
CREMA_train = CREMA_train.reset_index(drop=True) 
CREMA_val = CREMA_val.reset_index(drop = True) 

# Model

In [56]:
def create_model( init_mode='glorot_uniform', lr = 0.001, input_dim=(157, 13)):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=input_dim, kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

# Experiment 2.1 : RAVDESS

In [57]:
df_train = RAV_train
df_val = RAV_val
df_test = RAV_test

In [58]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [59]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13)

100%|████████████████████████████████████████| 120/120 [00:00<00:00, 934.97it/s]


In [60]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [61]:
np.size(y_val)

120

In [62]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [63]:
X_train.shape

(1200, 157, 13)

## Shuffle training data

In [64]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [65]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_1.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [91]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [94]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 11:57:58.203152: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 11:57:58.204291: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 11:57:58.590936: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 11:57:58.591586: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch 1/50
100/100 - 3s - loss: 1.2126 - accuracy: 0.5300
Epoch 1/50
100/100 - 3s - loss: 1.0144 - accuracy: 0.5512
Epoch 1/50
100/100 - 3s - loss: 0.8302 - accuracy: 0.6087
Epoch 1/50
100/100 - 4s - loss: 0.8718 - accuracy: 0.5838
Epoch 1/50
100/100 - 4s - loss: 1.1101 - accuracy: 0.5788
Epoch 1/50
100/100 - 4s - loss: 0.8419 - accuracy: 0.5562
Epoch 2/50
100/100 - 3s - loss: 0.6208 - accuracy: 0.6725
Epoch 2/50
100/100 - 3s - loss: 0.9552 - accuracy: 0.5813
Epoch 2/50
100/100 - 3s - loss: 0.6133 - accuracy: 0.6800
Epoch 2/50
100/100 - 3s - loss: 0.9151 - accuracy: 0.6313
Epoch 2/50
100/100 - 3s - loss: 0.8395 - accuracy: 0.5825
Epoch 2/50
100/100 - 3s - loss: 0.6398 - accuracy: 0.6637
Epoch 3/50
100/100 - 2s - loss: 0.8474 - accuracy: 0.6112
Epoch 3/50
100/100 - 3s - loss: 0.5769 - accuracy: 0.7175
Epoch 3/50
100/100 - 3s - loss: 0.5433 - accuracy: 0.7225
Epoch 3/50
100/100 - 3s - loss: 0.8427 - accuracy: 0.5825
Epoch 3/50
100/100 - 3s - loss: 0.6129 - accuracy: 0.6662
Epoch 3/50
100

2021-09-14 11:58:07.155868: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-14 11:58:07.157647: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1


Epoch 4/50
100/100 - 3s - loss: 0.6726 - accuracy: 0.6587


2021-09-14 11:58:07.413621: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-14 11:58:07.429492: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /proc/driver/nvidia/version does not exist
2021-09-14 11:58:07.430562: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-14 11:58:07.442076: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Epoch 4/50
100/100 - 3s - loss: 0.5556 - accuracy: 0.7025
Epoch 4/50
100/100 - 3s - loss: 0.5347 - accuracy: 0.7138
Epoch 4/50
100/100 - 3s - loss: 0.7080 - accuracy: 0.6475
Epoch 4/50
100/100 - 3s - loss: 0.5638 - accuracy: 0.7100


2021-09-14 11:58:08.417322: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-14 11:58:08.439492: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299965000 Hz


Epoch 4/50
100/100 - 3s - loss: 0.6699 - accuracy: 0.6475


2021-09-14 11:58:08.731605: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-14 11:58:08.752947: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-14 11:58:09.004422: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-14 11:58:09.005146: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /proc/driver/nvidia/version does not exist
2021-09-14 11:58:09.005689: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-0

Epoch 5/50
100/100 - 3s - loss: 0.7200 - accuracy: 0.6450
Epoch 5/50
100/100 - 3s - loss: 0.5336 - accuracy: 0.7312
Epoch 5/50
100/100 - 3s - loss: 0.5166 - accuracy: 0.7275
Epoch 5/50
100/100 - 3s - loss: 0.6928 - accuracy: 0.6325
Epoch 5/50
100/100 - 3s - loss: 0.5062 - accuracy: 0.7588
Epoch 5/50
100/100 - 3s - loss: 0.6814 - accuracy: 0.6525
Epoch 6/50
100/100 - 3s - loss: 0.6517 - accuracy: 0.6525
Epoch 6/50
100/100 - 3s - loss: 0.5168 - accuracy: 0.7387
Epoch 6/50
100/100 - 3s - loss: 0.4915 - accuracy: 0.7600
Epoch 6/50
100/100 - 2s - loss: 0.6003 - accuracy: 0.6875
Epoch 6/50
100/100 - 3s - loss: 0.6830 - accuracy: 0.6450
Epoch 6/50
100/100 - 3s - loss: 0.4772 - accuracy: 0.7725
Epoch 1/50
100/100 - 8s - loss: 2.9385 - accuracy: 0.5512
Epoch 7/50
100/100 - 4s - loss: 0.6181 - accuracy: 0.6700
Epoch 7/50
100/100 - 3s - loss: 0.4676 - accuracy: 0.7775
Epoch 7/50
100/100 - 4s - loss: 0.4510 - accuracy: 0.7788
Epoch 7/50
100/100 - 4s - loss: 0.5982 - accuracy: 0.6925
Epoch 7/50
100

In [96]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.7074999809265137 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6833, std=0.01359 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.7075, std=0.004082 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6575, std=0.01275 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.6867, std=0.02366 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.6967, std=0.01897 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.6758, std=0.01983 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6933, std=0.008498 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.6792, std=0.01929 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.705, std=0.007071 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.685, std=0.00736 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_s

## Train with best parameters

In [97]:
#Best Accuracy 0.7074999809265137 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
def create_model( init_mode='glorot_normal', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [98]:
seed = 7
np.random.seed(seed)

In [99]:
model = create_model()

In [100]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [101]:
import datetime, os

In [102]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [103]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 12:14:08.515010: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 12:14:08.515060: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 12:14:08.571728: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [104]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [105]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 32/150 [=====>........................] - ETA: 0s - loss: 1.2269 - accuracy: 0.5271

2021-09-14 12:14:09.447666: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 12:14:09.447692: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 12:14:09.457310: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 12:14:09.459972: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 12:14:09.463603: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-121403/train/plugins/profile/2021_09_14_12_14_09
2021-09-14 12:14:09.464337: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-121403/train/plugins/profile/2021_09_14_12_14_09/helemanc-Latitude-5410.trace.json.gz
2021-09-14 12:14:09.469821: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-121403/train/plugins/p

150/150 [==============================] - 1s 7ms/step - loss: 1.0147 - accuracy: 0.5686 - val_loss: 0.6090 - val_accuracy: 0.7250
Epoch 2/500
150/150 [==============================] - 1s 6ms/step - loss: 0.6412 - accuracy: 0.6417 - val_loss: 0.6714 - val_accuracy: 0.5417
Epoch 3/500
150/150 [==============================] - 1s 6ms/step - loss: 0.6070 - accuracy: 0.6748 - val_loss: 0.5554 - val_accuracy: 0.7833
Epoch 4/500
150/150 [==============================] - 1s 6ms/step - loss: 0.5732 - accuracy: 0.7010 - val_loss: 0.5384 - val_accuracy: 0.7083
Epoch 5/500
150/150 [==============================] - 1s 6ms/step - loss: 0.5896 - accuracy: 0.6874 - val_loss: 0.4924 - val_accuracy: 0.8083
Epoch 6/500
150/150 [==============================] - 1s 6ms/step - loss: 0.5099 - accuracy: 0.7609 - val_loss: 0.4851 - val_accuracy: 0.8000
Epoch 7/500
150/150 [==============================] - 1s 6ms/step - loss: 0.5432 - accuracy: 0.7099 - val_loss: 0.4953 - val_accuracy: 0.7500
Epoch 8/500

In [106]:
%tensorboard --logdir logs

In [107]:
model.evaluate(X_test, y_test, batch_size=8)

15/15 [==============================] - 0s 2ms/step - loss: 0.3263 - accuracy: 0.8167


[0.3262559473514557, 0.8166666626930237]

In [108]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.75      0.91      0.82        56
           1       0.90      0.73      0.81        64

    accuracy                           0.82       120
   macro avg       0.83      0.82      0.82       120
weighted avg       0.83      0.82      0.82       120



## Save best model 

In [109]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_1")

2021-09-14 12:18:06.835238: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_1/model_2_1/assets


# Experiment 2.2 : RAVDESS noise

## Read dataframes

In [66]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/ravdess"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [67]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13) # 13

100%|███████████████████████████████████████| 120/120 [00:00<00:00, 1170.85it/s]


In [68]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [69]:
np.size(y_val)

120

In [70]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [71]:
X_train.shape

(2400, 157, 13)

## Shuffle training data

In [72]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [73]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_2.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [116]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [117]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 12:27:07.003415: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 12:27:07.003644: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 12:27:07.079545: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 12:27:07.079704: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 12:27:07.122634: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
200/200 - 7s - loss: 1.2071 - accuracy: 0.5369
Epoch 1/50
200/200 - 7s - loss: 1.3629 - accuracy: 0.5119
Epoch 1/50
200/200 - 8s - loss: 0.8053 - accuracy: 0.5631
Epoch 1/50
200/200 - 8s - loss: 0.7927 - accuracy: 0.5688
Epoch 1/50
200/200 - 9s - loss: 1.0311 - accuracy: 0.5238
Epoch 1/50
200/200 - 9s - loss: 0.7825 - accuracy: 0.5794
Epoch 1/50
200/200 - 10s - loss: 2.8611 - accuracy: 0.5144
Epoch 1/50
200/200 - 10s - loss: 2.7691 - accuracy: 0.5044
Epoch 2/50
200/200 - 7s - loss: 0.8647 - accuracy: 0.5881
Epoch 2/50
200/200 - 7s - loss: 0.9170 - accuracy: 0.5800
Epoch 2/50
200/200 - 7s - loss: 0.6643 - accuracy: 0.6012
Epoch 2/50
200/200 - 7s - loss: 0.6735 - accuracy: 0.5875
Epoch 2/50
200/200 - 7s - loss: 0.6269 - accuracy: 0.6556
Epoch 2/50
200/200 - 8s - loss: 0.8238 - accuracy: 0.5462
Epoch 2/50
200/200 - 7s - loss: 1.7577 - accuracy: 0.5431
Epoch 2/50
200/200 - 7s - loss: 1.8108 - accuracy: 0.5525
Epoch 3/50
200/200 - 7s - loss: 0.7558 - accuracy: 0.5813
Epoch 3/50
2

In [119]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.8050000071525574 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
 mean=0.7254, std=0.04601 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.7887, std=0.03747 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6675, std=0.03704 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.715, std=0.02298 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.7925, std=0.03954 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.7271, std=0.02306 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6588, std=0.0398 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.7312, std=0.03482 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.7833, std=0.04288 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.805, std=0.03572 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}


## Train with best parameters

In [120]:
# Best Accuracy 0.8050000071525574 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}

def create_model( init_mode='uniform', lr = 0.0001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157, 12
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [121]:
seed = 7
np.random.seed(seed)

In [122]:
model = create_model()

In [123]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [124]:
import datetime, os

In [125]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [126]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 13:00:33.352387: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 13:00:33.352431: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 13:00:33.352474: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [127]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [128]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 32/600 [>.............................] - ETA: 3s - loss: 0.7644 - accuracy: 0.5415   

2021-09-14 13:00:37.534212: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 13:00:37.534233: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 13:00:37.586379: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 13:00:37.587129: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 13:00:37.588324: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-130033/train/plugins/profile/2021_09_14_13_00_37
2021-09-14 13:00:37.589023: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-130033/train/plugins/profile/2021_09_14_13_00_37/helemanc-Latitude-5410.trace.json.gz
2021-09-14 13:00:37.590091: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-130033/train/plugins/p

600/600 [==============================] - 3s 4ms/step - loss: 0.7065 - accuracy: 0.5726 - val_loss: 0.6255 - val_accuracy: 0.6500
Epoch 2/500
600/600 [==============================] - 2s 4ms/step - loss: 0.6455 - accuracy: 0.6058 - val_loss: 0.6019 - val_accuracy: 0.7917
Epoch 3/500
600/600 [==============================] - 2s 4ms/step - loss: 0.6235 - accuracy: 0.6424 - val_loss: 0.5740 - val_accuracy: 0.8250
Epoch 4/500
600/600 [==============================] - 2s 4ms/step - loss: 0.6001 - accuracy: 0.6605 - val_loss: 0.5369 - val_accuracy: 0.8167
Epoch 5/500
600/600 [==============================] - 2s 4ms/step - loss: 0.5698 - accuracy: 0.6954 - val_loss: 0.5066 - val_accuracy: 0.8333
Epoch 6/500
600/600 [==============================] - 2s 4ms/step - loss: 0.5804 - accuracy: 0.6949 - val_loss: 0.5170 - val_accuracy: 0.8083
Epoch 7/500
600/600 [==============================] - 2s 4ms/step - loss: 0.5449 - accuracy: 0.7106 - val_loss: 0.5180 - val_accuracy: 0.7917
Epoch 8/500

In [129]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 1:02:33 ago. (Use '!kill 6830' to kill it.)

In [130]:
model.evaluate(X_test, y_test, batch_size=4)

30/30 [==============================] - 0s 2ms/step - loss: 0.4322 - accuracy: 0.8000


[0.43224629759788513, 0.800000011920929]

In [131]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.72      0.93      0.81        56
           1       0.92      0.69      0.79        64

    accuracy                           0.80       120
   macro avg       0.82      0.81      0.80       120
weighted avg       0.83      0.80      0.80       120



## Save best model 

In [132]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_1/model_2_2")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_1/model_2_2/assets


# Experiment 2.3: TESS

In [74]:
df_train = TESS_train
df_test = TESS_test

In [75]:
df_train.reset_index(drop = True, inplace = True) 
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [76]:
X_train, y_train, X_test, y_test = feature_extractor_tess(df_train,  df_test, 13)

100%|█████████████████████████████████████| 1400/1400 [00:00<00:00, 1484.88it/s]


In [77]:
y_train, y_test = encode_labels_tess(y_train, y_test)

In [78]:
np.size(y_test)

1400

In [79]:
X_train, X_test, fitted_scaler = standard_scaling_tess(X_train,  X_test)

In [80]:
X_train.shape

(1400, 157, 13)

## Shuffle training data

In [81]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [82]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_3.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [143]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [144]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 13:32:35.943464: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 13:32:35.943712: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 13:32:35.989909: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 13:32:35.990182: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 13:32:36.076858: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
117/117 - 5s - loss: 0.8789 - accuracy: 0.6720
Epoch 1/50
117/117 - 5s - loss: 0.8234 - accuracy: 0.6613
Epoch 1/50
117/117 - 6s - loss: 0.2896 - accuracy: 0.8746
Epoch 1/50
117/117 - 6s - loss: 0.2497 - accuracy: 0.9133
Epoch 1/50
117/117 - 6s - loss: 0.4429 - accuracy: 0.8274
Epoch 1/50
117/117 - 6s - loss: 2.6892 - accuracy: 0.5991
Epoch 1/50
117/117 - 6s - loss: 0.6679 - accuracy: 0.7794
Epoch 1/50
117/117 - 6s - loss: 2.4881 - accuracy: 0.5959
Epoch 2/50
117/117 - 4s - loss: 0.4347 - accuracy: 0.8328
Epoch 2/50
117/117 - 5s - loss: 0.3608 - accuracy: 0.8499
Epoch 2/50
117/117 - 4s - loss: 0.0462 - accuracy: 0.9818
Epoch 2/50
117/117 - 4s - loss: 0.0404 - accuracy: 0.9839
Epoch 2/50
117/117 - 5s - loss: 0.0226 - accuracy: 0.9925
Epoch 2/50
117/117 - 4s - loss: 1.1865 - accuracy: 0.7717
Epoch 2/50
117/117 - 4s - loss: 0.2738 - accuracy: 0.8940
Epoch 2/50
117/117 - 4s - loss: 1.1843 - accuracy: 0.7567
Epoch 3/50
117/117 - 4s - loss: 0.3470 - accuracy: 0.8521
Epoch 3/50
117

In [146]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.6925938924153646 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.569, std=0.3487 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.6926, std=0.2793 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.5676, std=0.3454 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.6511, std=0.322 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.6147, std=0.344 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.6083, std=0.3417 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6025, std=0.3512 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.5747, std=0.3487 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.589, std=0.3514 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.6054, std=0.3296 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}


## Train with best parameters

In [147]:
#Best Accuracy 0.6925938924153646 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
def create_model( init_mode='glorot_normal', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [148]:
seed = 7
np.random.seed(seed)

In [149]:
model = create_model()

In [150]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [151]:
import datetime, os

In [152]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [153]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 13:53:50.765048: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 13:53:50.765079: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 13:53:50.765122: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [154]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [155]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500, 
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 23/175 [==>...........................] - ETA: 1s - loss: 1.1294 - accuracy: 0.5209

2021-09-14 13:53:53.640014: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 13:53:53.640038: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 13:53:53.698714: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 13:53:53.699512: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 13:53:53.700773: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-135349/train/plugins/profile/2021_09_14_13_53_53
2021-09-14 13:53:53.701505: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-135349/train/plugins/profile/2021_09_14_13_53_53/helemanc-Latitude-5410.trace.json.gz
2021-09-14 13:53:53.702578: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-135349/train/plugins/p

175/175 [==============================] - 1s 6ms/step - loss: 0.6491 - accuracy: 0.7422
Epoch 2/500
175/175 [==============================] - 1s 7ms/step - loss: 0.0348 - accuracy: 0.9892
Epoch 3/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0322 - accuracy: 0.9870
Epoch 4/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0260 - accuracy: 0.9894
Epoch 5/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0350 - accuracy: 0.9900
Epoch 6/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0045 - accuracy: 0.9993
Epoch 7/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0114 - accuracy: 0.9962
Epoch 8/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0587 - accuracy: 0.9877
Epoch 9/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0116 - accuracy: 0.9979
Epoch 10/500
175/175 [==============================] - 1s 6ms/step - loss: 0.0170 - accuracy: 0.9937



In [156]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 1:37:57 ago. (Use '!kill 6830' to kill it.)

In [157]:
model.evaluate(X_test, y_test, batch_size=8)

175/175 [==============================] - 0s 2ms/step - loss: 9.5421 - accuracy: 0.5643


[9.542071342468262, 0.5642856955528259]

In [158]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.19      0.01      0.01       600
           1       0.57      0.98      0.72       800

    accuracy                           0.56      1400
   macro avg       0.38      0.49      0.37      1400
weighted avg       0.41      0.56      0.42      1400



## Save best model 

In [159]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_3")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_3/assets


# Experiment 2.4: TESS noise

## Read dataframes

In [83]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/tess"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [84]:
X_train, y_train, X_test, y_test = feature_extractor_tess(df_train, df_test, 13) # 13

100%|█████████████████████████████████████| 1400/1400 [00:00<00:00, 1834.92it/s]


In [85]:
y_train,  y_test = encode_labels_tess(y_train,  y_test)

In [86]:
np.size(y_train)

2800

In [87]:
X_train, X_test, fitted_scaler = standard_scaling_tess(X_train,  X_test)

In [88]:
X_train.shape

(2800, 157, 13)

## Shuffle training data

In [89]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [90]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_4.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [165]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [166]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 15:02:56.268024: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 15:02:56.268024: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 15:02:56.268052: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 15:02:56.268294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 15:02:56.279490: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
234/234 - 9s - loss: 0.3439 - accuracy: 0.8495
Epoch 1/50
234/234 - 9s - loss: 0.8982 - accuracy: 0.6088
Epoch 1/50
234/234 - 9s - loss: 0.8794 - accuracy: 0.6588
Epoch 1/50
234/234 - 10s - loss: 0.7300 - accuracy: 0.7434
Epoch 1/50
234/234 - 10s - loss: 0.4013 - accuracy: 0.8226
Epoch 1/50
234/234 - 10s - loss: 2.1234 - accuracy: 0.5932
Epoch 1/50
234/234 - 10s - loss: 0.2177 - accuracy: 0.9127
Epoch 1/50
234/234 - 10s - loss: 2.2937 - accuracy: 0.6331
Epoch 2/50
234/234 - 8s - loss: 0.1395 - accuracy: 0.9454
Epoch 2/50
234/234 - 9s - loss: 0.4518 - accuracy: 0.7991
Epoch 2/50
234/234 - 9s - loss: 0.5034 - accuracy: 0.7594
Epoch 2/50
234/234 - 9s - loss: 0.2590 - accuracy: 0.8918
Epoch 2/50
234/234 - 9s - loss: 1.0490 - accuracy: 0.7122
Epoch 2/50
234/234 - 9s - loss: 0.0478 - accuracy: 0.9834
Epoch 2/50
234/234 - 8s - loss: 1.2222 - accuracy: 0.7258
Epoch 2/50
234/234 - 9s - loss: 0.1752 - accuracy: 0.9250
Epoch 3/50
234/234 - 9s - loss: 0.1091 - accuracy: 0.9625
Epoch 3/5

In [168]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.8681672016779581 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.8321, std=0.1675 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.8424, std=0.1475 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8542, std=0.1397 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.865, std=0.135 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.8682, std=0.127 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.8124, std=0.1514 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8382, std=0.1259 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8389, std=0.1433 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8217, std=0.1594 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.821, std=0.1676 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}


## Train with best parameters

In [169]:
#Best Accuracy 0.8681672016779581 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
def create_model( init_mode='uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157, 12
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [170]:
seed = 7
np.random.seed(seed)

In [171]:
model = create_model()

In [172]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [173]:
import datetime, os

In [174]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [175]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 15:36:35.858727: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 15:36:35.858760: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 15:36:35.858817: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [176]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [177]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500,
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 21/350 [>.............................] - ETA: 2s - loss: 0.8807 - accuracy: 0.5312

2021-09-14 15:36:42.864707: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 15:36:42.864732: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 15:36:42.922193: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 15:36:42.922989: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 15:36:42.924271: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-153635/train/plugins/profile/2021_09_14_15_36_42
2021-09-14 15:36:42.925075: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-153635/train/plugins/profile/2021_09_14_15_36_42/helemanc-Latitude-5410.trace.json.gz
2021-09-14 15:36:42.926140: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-153635/train/plugins/p

350/350 [==============================] - 2s 6ms/step - loss: 0.4356 - accuracy: 0.7917
Epoch 2/500
350/350 [==============================] - 2s 6ms/step - loss: 0.1173 - accuracy: 0.9572
Epoch 3/500
350/350 [==============================] - 2s 6ms/step - loss: 0.0895 - accuracy: 0.9684
Epoch 4/500
350/350 [==============================] - 2s 6ms/step - loss: 0.0727 - accuracy: 0.9710
Epoch 5/500
350/350 [==============================] - 2s 6ms/step - loss: 0.1003 - accuracy: 0.9668
Epoch 6/500
350/350 [==============================] - 2s 6ms/step - loss: 0.0409 - accuracy: 0.9843
Epoch 7/500
350/350 [==============================] - 2s 6ms/step - loss: 0.0464 - accuracy: 0.9801
Epoch 8/500
350/350 [==============================] - 2s 5ms/step - loss: 0.0413 - accuracy: 0.9818
Epoch 9/500
350/350 [==============================] - 2s 5ms/step - loss: 0.0527 - accuracy: 0.9824
Epoch 10/500
350/350 [==============================] - 2s 6ms/step - loss: 0.0447 - accuracy: 0.9826
E

In [178]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 3:23:18 ago. (Use '!kill 6830' to kill it.)

In [179]:
model.evaluate(X_test, y_test, batch_size=8)

175/175 [==============================] - 0s 2ms/step - loss: 11.0320 - accuracy: 0.5707


[11.032045364379883, 0.5707142949104309]

In [180]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       600
           1       0.57      1.00      0.73       800

    accuracy                           0.57      1400
   macro avg       0.29      0.50      0.36      1400
weighted avg       0.33      0.57      0.42      1400



## Save best model 

In [181]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_4")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_4/assets


# Experiment 2.5: SAVEE

In [91]:
df_train = SAVEE_train
df_val = SAVEE_val
df_test = SAVEE_test

In [92]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [93]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13)

100%|███████████████████████████████████████| 120/120 [00:00<00:00, 1064.25it/s]


In [94]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [95]:
np.size(y_val)

120

In [96]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [97]:
X_train.shape

(240, 157, 13)

## Shuffle training data

In [98]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [99]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_5.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [189]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [190]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 15:54:13.386206: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 15:54:13.386420: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 15:54:13.395833: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 15:54:13.397763: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 15:54:13.439629: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
20/20 - 1s - loss: 1.6440 - accuracy: 0.5063
Epoch 2/50
20/20 - 0s - loss: 1.6728 - accuracy: 0.4875
Epoch 1/50
20/20 - 1s - loss: 1.4575 - accuracy: 0.5312
Epoch 1/50
20/20 - 1s - loss: 1.3132 - accuracy: 0.5688
Epoch 2/50
20/20 - 0s - loss: 1.2107 - accuracy: 0.5500
Epoch 3/50
20/20 - 0s - loss: 1.3715 - accuracy: 0.4500
Epoch 2/50
20/20 - 0s - loss: 1.3495 - accuracy: 0.4875
Epoch 1/50
20/20 - 2s - loss: 1.5131 - accuracy: 0.5375
Epoch 3/50
20/20 - 0s - loss: 1.4690 - accuracy: 0.5250
Epoch 4/50
20/20 - 1s - loss: 1.1785 - accuracy: 0.5875
Epoch 3/50
20/20 - 1s - loss: 1.4560 - accuracy: 0.4625
Epoch 1/50
20/20 - 2s - loss: 1.4566 - accuracy: 0.5437
Epoch 4/50
20/20 - 1s - loss: 1.2739 - accuracy: 0.5562
Epoch 1/50
20/20 - 3s - loss: 4.4425 - accuracy: 0.5625
Epoch 2/50
20/20 - 1s - loss: 0.8213 - accuracy: 0.5813
Epoch 1/50
20/20 - 2s - loss: 1.0850 - accuracy: 0.5250
Epoch 5/50
20/20 - 1s - loss: 1.1790 - accuracy: 0.5813
Epoch 4/50
20/20 - 1s - loss: 1.1573 - accuracy:

In [191]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.5958333313465118 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.4958, std=0.1268 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.5333, std=0.05621 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.4917, std=0.1218 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.5333, std=0.1062 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.525, std=0.07706 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.4125, std=0.06124 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.4542, std=0.0598 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.5958, std=0.08559 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.4917, std=0.04125 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.5042, std=0.02946 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}


## Train with best parameters

In [192]:
# Best Accuracy 0.5958333313465118 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
def create_model( init_mode='he_normal', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [193]:
seed = 7
np.random.seed(seed)

In [194]:
model = create_model()

In [195]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [196]:
import datetime, os

In [197]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [198]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 15:58:31.719110: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 15:58:31.719137: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 15:58:31.719178: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [199]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [200]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
21/30 [====================>.........] - ETA: 0s - loss: 6.6673 - accuracy: 0.4831

2021-09-14 15:58:34.799955: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 15:58:34.799980: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 15:58:34.858721: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 15:58:34.859639: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 15:58:34.860921: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-155831/train/plugins/profile/2021_09_14_15_58_34
2021-09-14 15:58:34.861699: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-155831/train/plugins/profile/2021_09_14_15_58_34/helemanc-Latitude-5410.trace.json.gz
2021-09-14 15:58:34.862784: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-155831/train/plugins/p

30/30 [==============================] - 1s 14ms/step - loss: 5.9554 - accuracy: 0.4839 - val_loss: 0.8074 - val_accuracy: 0.4833
Epoch 2/500
30/30 [==============================] - 0s 7ms/step - loss: 2.3446 - accuracy: 0.5402 - val_loss: 0.7701 - val_accuracy: 0.4833
Epoch 3/500
30/30 [==============================] - 0s 7ms/step - loss: 1.2610 - accuracy: 0.5463 - val_loss: 0.7131 - val_accuracy: 0.5750
Epoch 4/500
30/30 [==============================] - 0s 7ms/step - loss: 1.0256 - accuracy: 0.5758 - val_loss: 0.6668 - val_accuracy: 0.6000
Epoch 5/500
30/30 [==============================] - 0s 7ms/step - loss: 0.9137 - accuracy: 0.5513 - val_loss: 0.6826 - val_accuracy: 0.5167
Epoch 6/500
30/30 [==============================] - 0s 7ms/step - loss: 0.6803 - accuracy: 0.6835 - val_loss: 0.6844 - val_accuracy: 0.5417
Epoch 7/500
30/30 [==============================] - 0s 7ms/step - loss: 0.8148 - accuracy: 0.5720 - val_loss: 0.6946 - val_accuracy: 0.5417
Epoch 8/500
30/30 [=====

In [201]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 3:41:38 ago. (Use '!kill 6830' to kill it.)

In [202]:
model.evaluate(X_test, y_test, batch_size=8)

15/15 [==============================] - 0s 2ms/step - loss: 0.7054 - accuracy: 0.5250


[0.7053889036178589, 0.5249999761581421]

In [203]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.51      0.93      0.66        60
           1       0.64      0.12      0.20        60

    accuracy                           0.53       120
   macro avg       0.58      0.53      0.43       120
weighted avg       0.58      0.53      0.43       120



## Save best model 

In [204]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_5")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_5/assets


# Experiment 2.6: SAVEE noise

## Read dataframes

In [100]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/savee"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [101]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13) # 13

100%|████████████████████████████████████████| 120/120 [00:00<00:00, 895.37it/s]


In [102]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [103]:
np.size(y_val)

120

In [104]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [105]:
X_train.shape

(480, 157, 13)

## Shuffle training data

In [106]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [107]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_6.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [211]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [212]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 16:01:58.874487: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 16:01:58.874679: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 16:01:58.938191: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 16:01:58.938317: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 16:01:58.985993: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
40/40 - 2s - loss: 1.5526 - accuracy: 0.5344
Epoch 1/50
40/40 - 2s - loss: 1.6613 - accuracy: 0.4625
Epoch 1/50
40/40 - 2s - loss: 1.0859 - accuracy: 0.5250
Epoch 1/50
40/40 - 2s - loss: 1.1220 - accuracy: 0.5156
Epoch 1/50
40/40 - 3s - loss: 1.8048 - accuracy: 0.4688
Epoch 1/50
40/40 - 3s - loss: 2.5185 - accuracy: 0.5188
Epoch 1/50
40/40 - 3s - loss: 4.2926 - accuracy: 0.4719
Epoch 2/50
40/40 - 2s - loss: 1.4200 - accuracy: 0.5031
Epoch 1/50
40/40 - 3s - loss: 1.2308 - accuracy: 0.4469
Epoch 2/50
40/40 - 2s - loss: 1.3781 - accuracy: 0.5406
Epoch 2/50
40/40 - 2s - loss: 0.7590 - accuracy: 0.5562
Epoch 2/50
40/40 - 1s - loss: 0.8001 - accuracy: 0.5469
Epoch 2/50
40/40 - 2s - loss: 1.4933 - accuracy: 0.4875
Epoch 2/50
40/40 - 2s - loss: 2.9959 - accuracy: 0.5125
Epoch 2/50
40/40 - 1s - loss: 3.7073 - accuracy: 0.5188
Epoch 3/50
40/40 - 2s - loss: 1.2233 - accuracy: 0.5250
Epoch 2/50
40/40 - 2s - loss: 0.7408 - accuracy: 0.5562
Epoch 3/50
40/40 - 2s - loss: 1.1344 - accuracy:

In [213]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.787500003973643 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.6396, std=0.03474 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.7521, std=0.03281 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.5875, std=0.02652 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.6208, std=0.08544 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.7875, std=0.01021 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.6167, std=0.06966 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.5979, std=0.02301 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.6208, std=0.02357 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.7083, std=0.04574 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.75, std=0.01021 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}


## Train with best parameters

In [214]:
#Best Accuracy 0.787500003973643 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
def create_model( init_mode='uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157, 12
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [215]:
seed = 7
np.random.seed(seed)

In [216]:
model = create_model()

In [217]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [218]:
import datetime, os

In [219]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [220]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 16:10:14.661411: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 16:10:14.661450: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 16:10:14.661506: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [221]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [222]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 31/120 [======>.......................] - ETA: 0s - loss: 1.1120 - accuracy: 0.5565

2021-09-14 16:10:17.422262: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 16:10:17.422288: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 16:10:17.476682: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 16:10:17.477493: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 16:10:17.478802: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-161014/train/plugins/profile/2021_09_14_16_10_17
2021-09-14 16:10:17.479598: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-161014/train/plugins/profile/2021_09_14_16_10_17/helemanc-Latitude-5410.trace.json.gz
2021-09-14 16:10:17.480706: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-161014/train/plugins/p

120/120 [==============================] - 1s 6ms/step - loss: 0.9626 - accuracy: 0.5456 - val_loss: 0.6749 - val_accuracy: 0.6083
Epoch 2/500
120/120 [==============================] - 1s 4ms/step - loss: 0.7387 - accuracy: 0.5562 - val_loss: 0.6887 - val_accuracy: 0.5833
Epoch 3/500
120/120 [==============================] - 0s 4ms/step - loss: 0.7029 - accuracy: 0.5992 - val_loss: 0.6764 - val_accuracy: 0.5250
Epoch 4/500
120/120 [==============================] - 1s 4ms/step - loss: 0.6822 - accuracy: 0.5699 - val_loss: 0.6559 - val_accuracy: 0.6333
Epoch 5/500
120/120 [==============================] - 1s 4ms/step - loss: 0.7138 - accuracy: 0.5508 - val_loss: 0.6601 - val_accuracy: 0.5583
Epoch 6/500
120/120 [==============================] - 0s 4ms/step - loss: 0.6801 - accuracy: 0.6302 - val_loss: 0.7790 - val_accuracy: 0.5000
Epoch 7/500
120/120 [==============================] - 1s 4ms/step - loss: 0.6272 - accuracy: 0.6614 - val_loss: 0.6620 - val_accuracy: 0.6417
Epoch 8/500

In [223]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 3:53:23 ago. (Use '!kill 6830' to kill it.)

In [224]:
model.evaluate(X_test, y_test, batch_size=4)

30/30 [==============================] - 0s 2ms/step - loss: 0.8594 - accuracy: 0.5250


[0.8594061136245728, 0.5249999761581421]

In [225]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.51      0.97      0.67        60
           1       0.71      0.08      0.15        60

    accuracy                           0.53       120
   macro avg       0.61      0.53      0.41       120
weighted avg       0.61      0.53      0.41       120



## Save best model 

In [226]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_6")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_6/assets


# Experiment 2.7: CREMA 

In [108]:
df_train = CREMA_train
df_val = CREMA_val
df_test = CREMA_test

In [109]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [110]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13)

100%|█████████████████████████████████████████| 60/60 [00:00<00:00, 1054.99it/s]


In [111]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [112]:
np.size(y_val)

60

In [113]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [114]:
X_train.shape

(320, 157, 13)

## Shuffle training data

In [115]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [116]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_7.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [234]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [235]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 16:15:22.108332: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 16:15:22.108560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 16:15:22.159262: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 16:15:22.159505: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 16:15:22.184613: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
27/27 - 2s - loss: 1.6312 - accuracy: 0.5258
Epoch 1/50
27/27 - 2s - loss: 1.6193 - accuracy: 0.4413
Epoch 2/50
27/27 - 0s - loss: 1.3430 - accuracy: 0.5915
Epoch 2/50
27/27 - 0s - loss: 1.2851 - accuracy: 0.5446
Epoch 3/50
27/27 - 1s - loss: 1.0586 - accuracy: 0.5540
Epoch 1/50
27/27 - 2s - loss: 1.5011 - accuracy: 0.5421
Epoch 1/50
27/27 - 3s - loss: 3.1032 - accuracy: 0.5869
Epoch 3/50
27/27 - 1s - loss: 1.2554 - accuracy: 0.5258
Epoch 1/50
27/27 - 3s - loss: 4.4136 - accuracy: 0.5587
Epoch 1/50
27/27 - 3s - loss: 1.3025 - accuracy: 0.5493
Epoch 4/50
27/27 - 1s - loss: 1.1350 - accuracy: 0.6009
Epoch 1/50
27/27 - 3s - loss: 1.2607 - accuracy: 0.5634
Epoch 1/50
27/27 - 3s - loss: 1.2801 - accuracy: 0.5467
Epoch 2/50
27/27 - 1s - loss: 1.2432 - accuracy: 0.5981
Epoch 2/50
27/27 - 1s - loss: 3.1483 - accuracy: 0.5305
Epoch 4/50
27/27 - 1s - loss: 1.1302 - accuracy: 0.5962
Epoch 2/50
27/27 - 1s - loss: 0.7285 - accuracy: 0.6244
Epoch 5/50
27/27 - 1s - loss: 0.8634 - accuracy:

In [236]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.8312173088391622 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.7874, std=0.02769 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.8125, std=0.02733 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.7939, std=0.0525 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.7907, std=0.02423 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.8312, std=0.008324 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.7781, std=0.01565 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6222, std=0.07216 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8218, std=0.02772 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.7999, std=0.03538 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.8124, std=0.01413 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4

## Train with best parameters

In [237]:
#Best Accuracy 0.8312173088391622 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
def create_model( init_mode='uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [238]:
seed = 7
np.random.seed(seed)

In [239]:
model = create_model()

In [240]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [241]:
import datetime, os

In [242]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [243]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 16:21:52.132502: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 16:21:52.132575: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 16:21:52.132697: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [244]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [245]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
21/40 [==============>...............] - ETA: 0s - loss: 0.8425 - accuracy: 0.6359

2021-09-14 16:21:59.116736: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 16:21:59.116766: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 16:21:59.171959: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 16:21:59.172779: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 16:21:59.174052: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-162151/train/plugins/profile/2021_09_14_16_21_59
2021-09-14 16:21:59.174779: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-162151/train/plugins/profile/2021_09_14_16_21_59/helemanc-Latitude-5410.trace.json.gz
2021-09-14 16:21:59.175866: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-162151/train/plugins/p

40/40 [==============================] - 1s 12ms/step - loss: 0.8888 - accuracy: 0.6106 - val_loss: 0.5604 - val_accuracy: 0.7500
Epoch 2/500
40/40 [==============================] - 0s 7ms/step - loss: 0.7872 - accuracy: 0.6763 - val_loss: 0.4779 - val_accuracy: 0.8167
Epoch 3/500
40/40 [==============================] - 0s 7ms/step - loss: 0.7065 - accuracy: 0.6040 - val_loss: 0.4168 - val_accuracy: 0.8333
Epoch 4/500
40/40 [==============================] - 0s 7ms/step - loss: 0.4738 - accuracy: 0.7788 - val_loss: 0.6381 - val_accuracy: 0.7167
Epoch 5/500
40/40 [==============================] - 0s 7ms/step - loss: 0.4535 - accuracy: 0.7936 - val_loss: 0.5063 - val_accuracy: 0.7500
Epoch 6/500
40/40 [==============================] - 0s 7ms/step - loss: 0.4728 - accuracy: 0.7749 - val_loss: 0.4584 - val_accuracy: 0.8333
Epoch 7/500
40/40 [==============================] - 0s 7ms/step - loss: 0.4676 - accuracy: 0.7676 - val_loss: 0.3431 - val_accuracy: 0.8500
Epoch 8/500
40/40 [=====

In [246]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 4:04:53 ago. (Use '!kill 6830' to kill it.)

In [247]:
model.evaluate(X_test, y_test, batch_size=8)

8/8 [==============================] - 0s 2ms/step - loss: 0.3250 - accuracy: 0.8333


[0.3250487148761749, 0.8333333134651184]

In [248]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.58      0.58      0.58        12
           1       0.90      0.90      0.90        48

    accuracy                           0.83        60
   macro avg       0.74      0.74      0.74        60
weighted avg       0.83      0.83      0.83        60



## Save best model 

In [249]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_7")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_7/assets


# Experiment 2.8: CREMA - noise

In [117]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/crema"
df_train = pd.read_csv(os.path.join(preprocess_path,"df_train.csv"))
df_val = pd.read_csv(os.path.join(preprocess_path,"df_val.csv"))
df_test = pd.read_csv(os.path.join(preprocess_path,"df_test.csv"))  

## Feature Extraction

In [118]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13) # 13

100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 977.19it/s]


In [119]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [120]:
np.size(y_val)

60

In [121]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [122]:
X_train.shape

(640, 157, 13)

## Shuffle training data

In [123]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [124]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_8.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [256]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [257]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-14 16:26:19.827457: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 16:26:19.827709: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 16:26:19.837103: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 16:26:19.837375: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-14 16:26:19.888307: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
54/54 - 2s - loss: 0.9990 - accuracy: 0.5610
Epoch 1/50
54/54 - 3s - loss: 1.6268 - accuracy: 0.5574
Epoch 1/50
54/54 - 3s - loss: 3.3733 - accuracy: 0.5540
Epoch 1/50
54/54 - 3s - loss: 1.4272 - accuracy: 0.5258
Epoch 1/50
54/54 - 3s - loss: 1.2881 - accuracy: 0.5504
Epoch 1/50
54/54 - 4s - loss: 1.8357 - accuracy: 0.4918
Epoch 1/50
54/54 - 4s - loss: 1.0977 - accuracy: 0.5948
Epoch 2/50
54/54 - 3s - loss: 0.8519 - accuracy: 0.6080
Epoch 1/50
54/54 - 4s - loss: 3.3132 - accuracy: 0.5621
Epoch 2/50
54/54 - 2s - loss: 1.3267 - accuracy: 0.5597
Epoch 2/50
54/54 - 2s - loss: 1.2103 - accuracy: 0.5892
Epoch 2/50
54/54 - 2s - loss: 1.1510 - accuracy: 0.6323
Epoch 2/50
54/54 - 2s - loss: 3.0747 - accuracy: 0.5446
Epoch 2/50
54/54 - 2s - loss: 0.6595 - accuracy: 0.6581
Epoch 2/50
54/54 - 2s - loss: 0.8431 - accuracy: 0.6300
Epoch 3/50
54/54 - 2s - loss: 0.6397 - accuracy: 0.6925
Epoch 2/50
54/54 - 2s - loss: 2.4890 - accuracy: 0.5761
Epoch 3/50
54/54 - 2s - loss: 1.2126 - accuracy:

In [258]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.8811957836151123 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.7469, std=0.03216 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.8547, std=0.0115 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.725, std=0.009148 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.8609, std=0.02933 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.8812, std=0.02483 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.7172, std=0.02736 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6205, std=0.07646 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.839, std=0.03743 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8047, std=0.006248 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.8531, std=0.01838 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}

## Train with best parameters

In [259]:
#Best Accuracy 0.8811957836151123 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
def create_model( init_mode='uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157, 12
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [260]:
seed = 7
np.random.seed(seed)

In [261]:
model = create_model()

In [262]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [263]:
import datetime, os

In [264]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [265]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-14 16:38:13.553036: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 16:38:13.553086: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 16:38:13.553176: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [266]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [267]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
21/80 [======>.......................] - ETA: 0s - loss: 0.8457 - accuracy: 0.5531

2021-09-14 16:38:16.057538: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-14 16:38:16.057562: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-14 16:38:16.120622: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-14 16:38:16.121566: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-14 16:38:16.123022: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-163813/train/plugins/profile/2021_09_14_16_38_16
2021-09-14 16:38:16.123828: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210914-163813/train/plugins/profile/2021_09_14_16_38_16/helemanc-Latitude-5410.trace.json.gz
2021-09-14 16:38:16.125152: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210914-163813/train/plugins/p

80/80 [==============================] - 1s 9ms/step - loss: 0.7991 - accuracy: 0.6016 - val_loss: 0.4987 - val_accuracy: 0.8167
Epoch 2/500
80/80 [==============================] - 0s 6ms/step - loss: 0.6280 - accuracy: 0.6747 - val_loss: 0.5005 - val_accuracy: 0.8667
Epoch 3/500
80/80 [==============================] - 0s 6ms/step - loss: 0.5431 - accuracy: 0.7600 - val_loss: 0.7442 - val_accuracy: 0.5000
Epoch 4/500
80/80 [==============================] - 1s 6ms/step - loss: 0.5489 - accuracy: 0.7048 - val_loss: 0.4685 - val_accuracy: 0.8500
Epoch 5/500
80/80 [==============================] - 1s 7ms/step - loss: 0.5108 - accuracy: 0.7416 - val_loss: 0.5837 - val_accuracy: 0.7167
Epoch 6/500
80/80 [==============================] - 0s 6ms/step - loss: 0.4927 - accuracy: 0.7288 - val_loss: 0.4226 - val_accuracy: 0.8500

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/500
80/80 [==============================] - 0s 6ms/step - loss: 0.4915 - acc

In [268]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 6830), started 4:21:23 ago. (Use '!kill 6830' to kill it.)

In [269]:
model.evaluate(X_test, y_test, batch_size=8)

8/8 [==============================] - 0s 2ms/step - loss: 0.3664 - accuracy: 0.8667


[0.3664077818393707, 0.8666666746139526]

In [270]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.70      0.58      0.64        12
           1       0.90      0.94      0.92        48

    accuracy                           0.87        60
   macro avg       0.80      0.76      0.78        60
weighted avg       0.86      0.87      0.86        60



## Save best model 

In [271]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_8")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_8/assets


# Experiment 2.9: RAVDESS - TESS - SAVEE

In [125]:
df_train = pd.concat([RAV_train, SAVEE_train, TESS_train])
df_val = pd.concat([RAV_val, SAVEE_val])
#df_test = pd.concat([RAV_test, SAVEE_test, TESS_test])
df_test = pd.concat([RAV_train, SAVEE_test])

In [126]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [127]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13)

100%|█████████████████████████████████████| 1320/1320 [00:00<00:00, 1715.82it/s]


In [128]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [129]:
np.size(y_test)

1320

In [130]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [131]:
X_train.shape

(2840, 157, 13)

## Shuffle training data

In [132]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [133]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_9.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [285]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [286]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-15 10:58:11.408782: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 10:58:11.409088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 10:58:11.439588: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 10:58:11.439843: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 10:58:11.467733: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

KeyboardInterrupt: 

In [282]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.6404710610707601 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.5898, std=0.02362 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.6025, std=0.03097 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.5993, std=0.02311 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.6039, std=0.0405 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.6405, std=0.04564 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.5993, std=0.01938 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6197, std=0.05692 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.6208, std=0.03639 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.6056, std=0.03494 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.6102, std=0.04939 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}

## Train with best parameters

In [287]:
# Best Accuracy 0.6404710610707601 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
def create_model( init_mode='uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [288]:
seed = 7
np.random.seed(seed)

In [289]:
model = create_model()

In [290]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [291]:
import datetime, os

In [292]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [293]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-15 10:58:27.483144: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 10:58:27.483190: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 10:58:27.483290: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [294]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [295]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 21/355 [>.............................] - ETA: 3s - loss: 1.0086 - accuracy: 0.4934

2021-09-15 10:58:36.067621: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 10:58:36.067645: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 10:58:36.128626: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-15 10:58:36.129458: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-15 10:58:36.130757: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-105824/train/plugins/profile/2021_09_15_10_58_36
2021-09-15 10:58:36.131606: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210915-105824/train/plugins/profile/2021_09_15_10_58_36/helemanc-Latitude-5410.trace.json.gz
2021-09-15 10:58:36.132767: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-105824/train/plugins/p

355/355 [==============================] - 3s 6ms/step - loss: 0.6602 - accuracy: 0.6548 - val_loss: 0.7571 - val_accuracy: 0.5583
Epoch 2/500
355/355 [==============================] - 2s 6ms/step - loss: 0.4069 - accuracy: 0.7890 - val_loss: 0.7030 - val_accuracy: 0.5958
Epoch 3/500
355/355 [==============================] - 2s 6ms/step - loss: 0.3547 - accuracy: 0.8211 - val_loss: 0.6437 - val_accuracy: 0.5958
Epoch 4/500
355/355 [==============================] - 2s 6ms/step - loss: 0.3364 - accuracy: 0.8079 - val_loss: 0.6168 - val_accuracy: 0.7000
Epoch 5/500
355/355 [==============================] - 2s 6ms/step - loss: 0.3243 - accuracy: 0.8252 - val_loss: 0.6279 - val_accuracy: 0.6708
Epoch 6/500
355/355 [==============================] - 2s 6ms/step - loss: 0.3097 - accuracy: 0.8345 - val_loss: 0.6049 - val_accuracy: 0.6875
Epoch 7/500
355/355 [==============================] - 2s 6ms/step - loss: 0.2876 - accuracy: 0.8502 - val_loss: 0.6742 - val_accuracy: 0.6458
Epoch 8/500

In [296]:
%tensorboard --logdir logs

In [297]:
model.evaluate(X_test, y_test, batch_size=8)

205/205 [==============================] - 0s 2ms/step - loss: 0.6930 - accuracy: 0.5689


[0.6929938793182373, 0.5689024329185486]

In [309]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85       620
           1       0.92      0.77      0.84       700

    accuracy                           0.84      1320
   macro avg       0.85      0.85      0.84      1320
weighted avg       0.86      0.84      0.84      1320



## Save best model 

In [299]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_9")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_9/assets


# Experiment 2.10: RAVDESS - TESS - SAVEE noise

## Read dataframes

In [134]:
preprocess_path_rav = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/ravdess"
preprocess_path_savee = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/savee"
preprocess_path_tess = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/tess"

df_train_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_train.csv"))
df_val_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_val.csv"))
df_test_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_test.csv"))  

df_train_tess = pd.read_csv(os.path.join(preprocess_path_tess,"df_train.csv"))
df_test_tess= pd.read_csv(os.path.join(preprocess_path_tess,"df_test.csv"))  

df_train_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_train.csv"))
df_val_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_val.csv"))
df_test_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_test.csv"))  

In [135]:
df_train = pd.concat([df_train_rav, df_train_savee, df_train_tess])
df_val = pd.concat([df_val_rav, df_val_savee])
#df_test = pd.concat([df_test_rav, df_test_savee, df_test_tess])
df_test = pd.concat([RAV_train, SAVEE_test])

In [136]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [137]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13) # 13

100%|██████████████████████████████████████| 1320/1320 [00:02<00:00, 610.36it/s]


In [138]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [139]:
np.size(y_val)

240

In [140]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [141]:
X_train.shape

(5680, 157, 13)

## Shuffle training data

In [142]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [143]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_10.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [317]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [318]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-15 11:17:07.352209: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 11:17:07.352420: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 11:17:07.379747: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 11:17:07.379994: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 11:17:07.503218: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
237/237 - 10s - loss: 2.8735 - accuracy: 0.5520
Epoch 1/50
237/237 - 10s - loss: 0.6676 - accuracy: 0.6751
Epoch 1/50
237/237 - 11s - loss: 1.1132 - accuracy: 0.5755
Epoch 1/50
237/237 - 11s - loss: 1.2053 - accuracy: 0.5610
Epoch 1/50
237/237 - 11s - loss: 0.6882 - accuracy: 0.6487
Epoch 1/50
237/237 - 11s - loss: 1.1686 - accuracy: 0.5552
Epoch 1/50
237/237 - 12s - loss: 0.6911 - accuracy: 0.6595
Epoch 1/50
237/237 - 12s - loss: 3.0608 - accuracy: 0.5351
Epoch 2/50
237/237 - 10s - loss: 1.6948 - accuracy: 0.6154
Epoch 2/50
237/237 - 10s - loss: 0.4520 - accuracy: 0.7623
Epoch 2/50
237/237 - 10s - loss: 0.7541 - accuracy: 0.6362
Epoch 2/50
237/237 - 9s - loss: 0.4561 - accuracy: 0.7538
Epoch 2/50
237/237 - 10s - loss: 0.7466 - accuracy: 0.6371
Epoch 2/50
237/237 - 9s - loss: 0.7275 - accuracy: 0.6566
Epoch 2/50
237/237 - 9s - loss: 0.4580 - accuracy: 0.7672
Epoch 2/50
237/237 - 9s - loss: 1.8932 - accuracy: 0.5964
Epoch 3/50
237/237 - 9s - loss: 0.3966 - accuracy: 0.7961
Ep

In [319]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.9042315085728964 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
 mean=0.8873, std=0.0185 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.9004, std=0.01291 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8623, std=0.005479 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.8785, std=0.01692 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.9035, std=0.01463 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.8641, std=0.003829 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8497, std=0.01903 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8838, std=0.005604 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8912, std=0.005613 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.9042, std=0.01267 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size

## Train with best parameters

In [320]:
#Best Accuracy 0.9042315085728964 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
def create_model( init_mode='uniform', lr = 0.0001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157, 12
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [321]:
seed = 7
np.random.seed(seed)

In [322]:
model = create_model()

In [323]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [324]:
import datetime, os

In [325]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [326]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-15 11:53:53.480287: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 11:53:53.480334: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 11:53:53.480428: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [327]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [328]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 20/710 [..............................] - ETA: 7s - loss: 0.6778 - accuracy: 0.5966

2021-09-15 11:53:59.821878: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 11:53:59.821917: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 11:53:59.887882: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-15 11:53:59.888732: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-15 11:53:59.890160: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-115352/train/plugins/profile/2021_09_15_11_53_59
2021-09-15 11:53:59.890901: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210915-115352/train/plugins/profile/2021_09_15_11_53_59/helemanc-Latitude-5410.trace.json.gz
2021-09-15 11:53:59.892049: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-115352/train/plugins/p

710/710 [==============================] - 5s 7ms/step - loss: 0.6764 - accuracy: 0.5826 - val_loss: 0.6704 - val_accuracy: 0.6542
Epoch 2/500
710/710 [==============================] - 4s 6ms/step - loss: 0.5249 - accuracy: 0.7298 - val_loss: 0.6518 - val_accuracy: 0.6417
Epoch 3/500
710/710 [==============================] - 4s 6ms/step - loss: 0.4473 - accuracy: 0.7751 - val_loss: 0.6346 - val_accuracy: 0.6208
Epoch 4/500
710/710 [==============================] - 4s 6ms/step - loss: 0.3782 - accuracy: 0.8212 - val_loss: 0.6183 - val_accuracy: 0.6958
Epoch 5/500
710/710 [==============================] - 4s 6ms/step - loss: 0.3675 - accuracy: 0.8168 - val_loss: 0.6147 - val_accuracy: 0.6750
Epoch 6/500
710/710 [==============================] - 4s 6ms/step - loss: 0.3475 - accuracy: 0.8190 - val_loss: 0.6231 - val_accuracy: 0.6042
Epoch 7/500
710/710 [==============================] - 4s 6ms/step - loss: 0.3283 - accuracy: 0.8310 - val_loss: 0.6073 - val_accuracy: 0.6292
Epoch 8/500

In [329]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 21930), started 0:56:15 ago. (Use '!kill 21930' to kill it.)

In [345]:
model.evaluate(X_test, y_test, batch_size=4)

330/330 [==============================] - 0s 1ms/step - loss: 0.3897 - accuracy: 0.8826


[0.3896549642086029, 0.8825757503509521]

In [346]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.86      0.89      0.88       620
           1       0.90      0.87      0.89       700

    accuracy                           0.88      1320
   macro avg       0.88      0.88      0.88      1320
weighted avg       0.88      0.88      0.88      1320



## Save best model 

In [332]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_10")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_10/assets


# Experiment 2.11: RAVDESS - TESS - SAVEE - CREMA

In [144]:
df_train = pd.concat([RAV_train, SAVEE_train, TESS_train, CREMA_train])
df_val = pd.concat([RAV_val, SAVEE_val, CREMA_val])
#df_test = pd.concat([RAV_test, SAVEE_test, TESS_test, CREMA_test])
df_test = pd.concat([RAV_train, SAVEE_test])

In [145]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [146]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13)

100%|█████████████████████████████████████| 1320/1320 [00:01<00:00, 1296.77it/s]


In [147]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [148]:
np.size(y_test)

1320

In [149]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [150]:
X_train.shape

(3160, 157, 13)

## Shuffle training data

In [151]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [152]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_11.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [355]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [356]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-15 12:07:40.941889: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 12:07:40.941922: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 12:07:40.958537: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 12:07:40.958699: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 12:07:40.958919: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
264/264 - 11s - loss: 1.1058 - accuracy: 0.5596
Epoch 1/50
264/264 - 11s - loss: 0.6871 - accuracy: 0.6562
Epoch 1/50
264/264 - 11s - loss: 1.0635 - accuracy: 0.5852
Epoch 1/50
264/264 - 11s - loss: 1.0537 - accuracy: 0.5655
Epoch 1/50
264/264 - 11s - loss: 0.7068 - accuracy: 0.6474
Epoch 1/50
264/264 - 11s - loss: 3.0484 - accuracy: 0.5629
Epoch 1/50
264/264 - 11s - loss: 2.4103 - accuracy: 0.5589
Epoch 1/50
264/264 - 11s - loss: 0.6695 - accuracy: 0.6687
Epoch 2/50
264/264 - 10s - loss: 0.4791 - accuracy: 0.7441
Epoch 2/50
264/264 - 10s - loss: 0.4648 - accuracy: 0.7508
Epoch 2/50
264/264 - 10s - loss: 0.7453 - accuracy: 0.6450
Epoch 2/50
264/264 - 10s - loss: 0.7036 - accuracy: 0.6654
Epoch 2/50
264/264 - 10s - loss: 0.7557 - accuracy: 0.6263
Epoch 2/50
264/264 - 10s - loss: 1.8156 - accuracy: 0.6184
Epoch 2/50
264/264 - 10s - loss: 0.4674 - accuracy: 0.7622
Epoch 2/50
264/264 - 10s - loss: 1.4249 - accuracy: 0.6220
Epoch 3/50
264/264 - 10s - loss: 0.4201 - accuracy: 0.77

In [372]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.8876572251319885 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.8785, std=0.002278 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.8854, std=0.003698 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8481, std=0.0128 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.8506, std=0.01247 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.8867, std=0.005451 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.8579, std=0.005931 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8367, std=0.006399 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8617, std=0.007385 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8877, std=0.002287 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.8801, std=0.01279 using {'lr': 0.0001, 'init_mode': 'uniform', 'b

## Train with best parameters

In [359]:
#Best Accuracy 0.8876572251319885 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
def create_model( init_mode='glorot_normal', lr = 0.0001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [360]:
seed = 7
np.random.seed(seed)

In [361]:
model = create_model()

In [362]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [363]:
import datetime, os

In [364]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [365]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-15 12:48:39.955567: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 12:48:39.955650: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 12:48:39.955793: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [366]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [367]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 32/790 [>.............................] - ETA: 4s - loss: 0.9309 - accuracy: 0.5889

2021-09-15 12:48:44.155830: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 12:48:44.155852: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 12:48:44.224738: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-15 12:48:44.225641: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-15 12:48:44.226969: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-124839/train/plugins/profile/2021_09_15_12_48_44
2021-09-15 12:48:44.227782: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210915-124839/train/plugins/profile/2021_09_15_12_48_44/helemanc-Latitude-5410.trace.json.gz
2021-09-15 12:48:44.228971: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-124839/train/plugins/p

790/790 [==============================] - 4s 4ms/step - loss: 0.8242 - accuracy: 0.5669 - val_loss: 0.6840 - val_accuracy: 0.5767
Epoch 2/500
790/790 [==============================] - 3s 4ms/step - loss: 0.5771 - accuracy: 0.6967 - val_loss: 0.6627 - val_accuracy: 0.6167
Epoch 3/500
790/790 [==============================] - 3s 4ms/step - loss: 0.4906 - accuracy: 0.7527 - val_loss: 0.6452 - val_accuracy: 0.6533
Epoch 4/500
790/790 [==============================] - 3s 4ms/step - loss: 0.4383 - accuracy: 0.7830 - val_loss: 0.6247 - val_accuracy: 0.6900
Epoch 5/500
790/790 [==============================] - 3s 4ms/step - loss: 0.3963 - accuracy: 0.8010 - val_loss: 0.6188 - val_accuracy: 0.7367
Epoch 6/500
790/790 [==============================] - 3s 4ms/step - loss: 0.3728 - accuracy: 0.8038 - val_loss: 0.6142 - val_accuracy: 0.6800
Epoch 7/500
790/790 [==============================] - 3s 4ms/step - loss: 0.3634 - accuracy: 0.8175 - val_loss: 0.6032 - val_accuracy: 0.6933
Epoch 8/500

In [368]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 21930), started 2:03:08 ago. (Use '!kill 21930' to kill it.)

In [369]:
model.evaluate(X_test, y_test, batch_size=4)

425/425 [==============================] - 1s 1ms/step - loss: 0.6713 - accuracy: 0.6029


[0.6712924242019653, 0.6029411554336548]

In [380]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.77      0.95      0.85       620
           1       0.94      0.75      0.84       700

    accuracy                           0.84      1320
   macro avg       0.86      0.85      0.84      1320
weighted avg       0.86      0.84      0.84      1320



## Save best model 

In [371]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_11")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_11/assets


# Experiment 2.12:  RAVDESS - TESS - SAVEE - CREMA noise

## Read dataframes

In [153]:
preprocess_path_rav = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/ravdess"
preprocess_path_savee = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/savee"
preprocess_path_tess = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/tess"
preprocess_path_crema = "/home/helemanc/Desktop/Binary_Model/df_csv_noise/crema"

df_train_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_train.csv"))
df_val_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_val.csv"))
df_test_rav = pd.read_csv(os.path.join(preprocess_path_rav,"df_test.csv"))  

df_train_tess = pd.read_csv(os.path.join(preprocess_path_tess,"df_train.csv"))
df_test_tess= pd.read_csv(os.path.join(preprocess_path_tess,"df_test.csv"))  

df_train_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_train.csv"))
df_val_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_val.csv"))
df_test_savee = pd.read_csv(os.path.join(preprocess_path_savee,"df_test.csv"))  

df_train_crema = pd.read_csv(os.path.join(preprocess_path_crema,"df_train.csv"))
df_val_crema = pd.read_csv(os.path.join(preprocess_path_crema,"df_val.csv"))
df_test_crema = pd.read_csv(os.path.join(preprocess_path_crema,"df_test.csv")) 

In [154]:
df_train = pd.concat([df_train_rav, df_train_savee, df_train_tess, df_train_crema])
df_val = pd.concat([df_val_rav, df_val_savee, df_train_crema, df_train_crema])
#df_test = pd.concat([df_test_rav, df_test_savee, df_test_tess])
#df_test = pd.concat([df_test_rav, df_test_savee, df_test_crema ])
df_test = pd.concat([df_test_rav, df_test_savee ])

In [155]:
df_train.reset_index(drop = True, inplace = True) 
df_val.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

## Feature Extraction

In [156]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_extractor(df_train, df_val, df_test, 13) # 13

100%|███████████████████████████████████████| 240/240 [00:00<00:00, 1090.45it/s]


In [157]:
y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

In [158]:
np.size(y_val)

1520

In [159]:
X_train, X_val, X_test, fitted_scaler = standard_scaling(X_train, X_val, X_test)

In [160]:
X_train.shape

(6320, 157, 13)

## Shuffle training data

In [161]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Save Scaler

In [162]:
pkl_filename = "/home/helemanc/Desktop/Binary_Model/scalers_experiments/Experiment_2/scaler_2_12.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(fitted_scaler, file)

## Hypeparameter optimization

In [390]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [391]:
%%time

# set reproducibility 
seed = 7
np.random.seed(seed)

batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 'he_normal', 'he_uniform']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
#grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)
grid_result =  grid.fit(X_train, y_train,class_weight = class_weights)

2021-09-15 13:22:53.715225: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 13:22:53.715828: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 13:22:53.730261: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-15 13:22:53.730393: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-15 13:22:53.773895: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
527/527 - 20s - loss: 0.9566 - accuracy: 0.5659
Epoch 1/50
527/527 - 20s - loss: 1.9564 - accuracy: 0.5502
Epoch 1/50
527/527 - 21s - loss: 0.9827 - accuracy: 0.5754
Epoch 1/50
527/527 - 21s - loss: 2.3535 - accuracy: 0.5447
Epoch 1/50
527/527 - 21s - loss: 0.6694 - accuracy: 0.6423
Epoch 1/50
527/527 - 21s - loss: 0.9717 - accuracy: 0.5679
Epoch 1/50
527/527 - 21s - loss: 0.6522 - accuracy: 0.6314
Epoch 1/50
527/527 - 21s - loss: 0.6560 - accuracy: 0.6357
Epoch 2/50
527/527 - 20s - loss: 1.0654 - accuracy: 0.6048
Epoch 2/50
527/527 - 19s - loss: 0.6683 - accuracy: 0.6357
Epoch 2/50
527/527 - 20s - loss: 0.9777 - accuracy: 0.6069
Epoch 2/50
527/527 - 19s - loss: 0.5157 - accuracy: 0.7073
Epoch 2/50
527/527 - 21s - loss: 0.6643 - accuracy: 0.6235
Epoch 2/50
527/527 - 21s - loss: 0.6672 - accuracy: 0.6397
Epoch 2/50
527/527 - 21s - loss: 0.5052 - accuracy: 0.7105
Epoch 2/50
527/527 - 21s - loss: 0.5071 - accuracy: 0.7000
Epoch 3/50
527/527 - 19s - loss: 0.7165 - accuracy: 0.62

2021-09-15 13:40:10.878039: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17201548 exceeds 10% of free system memory.


Epoch 50/50
527/527 - 22s - loss: 0.3456 - accuracy: 0.8139
Epoch 50/50
527/527 - 23s - loss: 0.3557 - accuracy: 0.8134


2021-09-15 13:40:14.553742: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17201548 exceeds 10% of free system memory.
2021-09-15 13:40:14.706372: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17201548 exceeds 10% of free system memory.


...

Epoch 6/50
527/527 - 17s - loss: 0.6738 - accuracy: 0.6507
Epoch 7/50
527/527 - 17s - loss: 0.6395 - accuracy: 0.6494
Epoch 7/50
527/527 - 17s - loss: 0.6624 - accuracy: 0.6347
Epoch 8/50
527/527 - 19s - loss: 0.4893 - accuracy: 0.7383
Epoch 8/50
527/527 - 19s - loss: 0.4989 - accuracy: 0.7432
Epoch 1/50
527/527 - 25s - loss: 1.2147 - accuracy: 0.5756
Epoch 1/50
527/527 - 25s - loss: 1.2498 - accuracy: 0.6000
Epoch 7/50
527/527 - 21s - loss: 0.6293 - accuracy: 0.6626
Epoch 8/50
527/527 - 21s - loss: 0.6115 - accuracy: 0.6584
Epoch 1/50
527/527 - 27s - loss: 1.2179 - accuracy: 0.6009
Epoch 8/50
527/527 - 21s - loss: 0.6463 - accuracy: 0.6411
Epoch 9/50
527/527 - 21s - loss: 0.4832 - accuracy: 0.7456
Epoch 9/50
527/527 - 21s - loss: 0.4789 - accuracy: 0.7536
Epoch 2/50
527/527 - 21s - loss: 0.6225 - accuracy: 0.6176
Epoch 2/50
527/527 - 21s - loss: 0.5932 - accuracy: 0.6430
Epoch 8/50
527/527 - 20s - loss: 0.6277 - accuracy: 0.6626
Epoch 9/50
527/527 - 22s - loss: 0.6040 - accuracy: 0.66

2021-09-15 14:15:56.863735: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 34394932 exceeds 10% of free system memory.


Epoch 50/50
527/527 - 18s - loss: 0.3945 - accuracy: 0.7850
Epoch 43/50
527/527 - 18s - loss: 0.3104 - accuracy: 0.8294
Epoch 50/50
527/527 - 20s - loss: 0.4046 - accuracy: 0.7864
264/264 - 4s - loss: 0.4167 - accuracy: 0.7627
Epoch 50/50
527/527 - 15s - loss: 0.3850 - accuracy: 0.7978
Epoch 44/50
527/527 - 19s - loss: 0.3158 - accuracy: 0.8196
264/264 - 4s - loss: 0.4022 - accuracy: 0.8064


2021-09-15 14:16:10.972844: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 34394932 exceeds 10% of free system memory.


264/264 - 4s - loss: 0.4043 - accuracy: 0.8196
Epoch 44/50
527/527 - 18s - loss: 0.3285 - accuracy: 0.8127
Epoch 1/50
1054/1054 - 25s - loss: 0.7267 - accuracy: 0.5925
Epoch 44/50
527/527 - 18s - loss: 0.3062 - accuracy: 0.8339
Epoch 1/50
1054/1054 - 27s - loss: 0.7321 - accuracy: 0.5993
Epoch 45/50
527/527 - 19s - loss: 0.3026 - accuracy: 0.8293
Epoch 45/50
527/527 - 21s - loss: 0.3290 - accuracy: 0.8049
Epoch 1/50
1054/1054 - 28s - loss: 0.7424 - accuracy: 0.5990
Epoch 1/50
1054/1054 - 30s - loss: 0.6561 - accuracy: 0.6036
Epoch 45/50
527/527 - 22s - loss: 0.3029 - accuracy: 0.8386
Epoch 1/50
1054/1054 - 32s - loss: 0.6573 - accuracy: 0.6110
Epoch 2/50
1054/1054 - 26s - loss: 0.5906 - accuracy: 0.6689
Epoch 2/50
1054/1054 - 25s - loss: 0.5886 - accuracy: 0.6819
Epoch 46/50
527/527 - 22s - loss: 0.2997 - accuracy: 0.8244
Epoch 46/50
527/527 - 23s - loss: 0.3248 - accuracy: 0.8137
Epoch 2/50
1054/1054 - 24s - loss: 0.5851 - accuracy: 0.6749
Epoch 46/50
527/527 - 22s - loss: 0.3131 - ac

In [392]:
# print results
print(f'Best Accuracy {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy 0.8767395615577698 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
 mean=0.8476, std=0.004334 using {'lr': 0.0001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.8544, std=0.01044 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.8215, std=0.01173 using {'lr': 0.0001, 'init_mode': 'he_uniform', 'batch_size': 8}
 mean=0.782, std=0.006255 using {'lr': 0.001, 'init_mode': 'he_uniform', 'batch_size': 4}
 mean=0.8573, std=0.007963 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.8399, std=0.0006261 using {'lr': 5e-05, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.7962, std=0.0243 using {'lr': 5e-05, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8117, std=0.02464 using {'lr': 0.001, 'init_mode': 'he_normal', 'batch_size': 8}
 mean=0.8722, std=0.01239 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.8767, std=0.005825 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_siz

## Train with best parameters

In [393]:
#Best Accuracy 0.8767395615577698 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
def create_model( init_mode='uniform', lr = 0.0001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(157,13), kernel_initializer=init_mode)) # 157, 12
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6)) #0.6

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [394]:
seed = 7
np.random.seed(seed)

In [395]:
model = create_model()

In [396]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

In [397]:
import datetime, os

In [398]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [399]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-09-15 14:50:11.771569: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 14:50:11.771598: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 14:50:11.771652: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [400]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [401]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
  33/1580 [..............................] - ETA: 8s - loss: 0.8682 - accuracy: 0.5471 

2021-09-15 14:50:14.314576: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-09-15 14:50:14.314598: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-09-15 14:50:14.372625: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-09-15 14:50:14.373446: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-09-15 14:50:14.374680: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-145011/train/plugins/profile/2021_09_15_14_50_14
2021-09-15 14:50:14.375375: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210915-145011/train/plugins/profile/2021_09_15_14_50_14/helemanc-Latitude-5410.trace.json.gz
2021-09-15 14:50:14.376509: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210915-145011/train/plugins/p

1580/1580 [==============================] - 7s 4ms/step - loss: 0.6880 - accuracy: 0.5942 - val_loss: 0.6432 - val_accuracy: 0.5941
Epoch 2/500
1580/1580 [==============================] - 6s 4ms/step - loss: 0.5371 - accuracy: 0.7064 - val_loss: 0.6151 - val_accuracy: 0.6645
Epoch 3/500
1580/1580 [==============================] - 6s 4ms/step - loss: 0.4695 - accuracy: 0.7501 - val_loss: 0.6081 - val_accuracy: 0.7487
Epoch 4/500
1580/1580 [==============================] - 6s 4ms/step - loss: 0.4427 - accuracy: 0.7592 - val_loss: 0.5736 - val_accuracy: 0.7638
Epoch 5/500
1580/1580 [==============================] - 6s 4ms/step - loss: 0.4164 - accuracy: 0.7832 - val_loss: 0.6112 - val_accuracy: 0.7243
Epoch 6/500
1580/1580 [==============================] - 6s 4ms/step - loss: 0.3909 - accuracy: 0.7868 - val_loss: 0.5641 - val_accuracy: 0.7599
Epoch 7/500
1580/1580 [==============================] - 6s 4ms/step - loss: 0.3675 - accuracy: 0.8120 - val_loss: 0.5591 - val_accuracy: 0.75

In [402]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 21930), started 4:42:22 ago. (Use '!kill 21930' to kill it.)

In [417]:
model.evaluate(X_test, y_test, batch_size=4)

75/75 [==============================] - 0s 1ms/step - loss: 0.5834 - accuracy: 0.7267


[0.5834195017814636, 0.7266666889190674]

In [427]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.65      0.75      0.70       116
           1       0.73      0.63      0.68       124

    accuracy                           0.69       240
   macro avg       0.69      0.69      0.69       240
weighted avg       0.69      0.69      0.69       240



## Save best model 

In [405]:
model.save("/home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_12")

INFO:tensorflow:Assets written to: /home/helemanc/Desktop/Binary_Model/models_experiments/Experiment_2/model_2_12/assets
